# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [2]:
blogs = pd.read_csv("blogs.tsv", sep="\t", header=0)
blogs

,type,venue,url_slug,date,tags,title,talk_url,description
0,Blog,[Netherlands eScience Center](https://medium.c...,grlc_story,2024-10-24,"grlc,research software,story",From Corridor Coincidences to Cellular Cartogr...,https://blog.esciencecenter.nl/from-corridor-c...,This is a short story of how [grlc](https://gi...
1,Blog,[Netherlands eScience Center](https://medium.c...,beyond_research,2024-10-30,"story,personal",Beyond research,https://blog.esciencecenter.nl/beyond-research...,"In this blog, some and my colleagues from the ..."
2,Blog,[Netherlands eScience Center](https://medium.c...,NRSD_infrastructures,2024-09-19,"research software,national infrastructure",National Research Software Day: National Infra...,https://blog.esciencecenter.nl/national-resear...,This blog is a summary of the National Infrast...
3,Blog,[Netherlands eScience Center](https://medium.c...,nl_research_software_policy,2023-11-28,"research software,policy",Navigating research software policies: Insight...,https://medium.com/escience-center/navigating-...,This post is a summary of an event discussing ...
4,Blog,[Netherlands eScience Center](https://medium.c...,future_research_software,2023-11-23,"research software,policy",The Future of Research Software,https://medium.com/escience-center/the-future-...,This post highlights the key points from the [...
5,Blog,[Netherlands eScience Center](https://medium.c...,launch_smp_guidelines,2022-10-27,"research software,software management",How to manage your software,https://medium.com/escience-center/how-to-mana...,This blog was published to mark the release of...
6,Recommendations,[Science Europe](https://scienceeurope.org/),se_research_software,2024-09-10,"research software,policy",Developing and Aligning Policies on Research S...,https://scienceeurope.org/our-resources/recomm...,I helped in the creation of [these](https://do...
7,Podcast,[Sustain OSS](https://podcast.sustainoss.org/),sustain_oss_wosss,2023-12-15,"research software,software sustainability",Episode 212: Workshop on Sustainable Software ...,https://podcast.sustainoss.org/212,"In this episode, Richards welcomes his two gue..."


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [3]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [4]:
loc_dict = {}

for row, item in blogs.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: blogs" + "\n"

    # TODO: Not needed in TSV -- or use in template?
    # if len(str(item.type)) > 3:
    #     md += 'type: "' + item.type + '"\n'
    # else:
    #     md += 'type: "Talk"\n'
    
    md += "permalink: /blogs/" + html_filename + "\n"

    # Venue is where the blog was published
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.date)) > 3:
        md += "date: " + str(item.date) + "\n"

    # TODO: Not needed in TSV?
    # if len(str(item.location)) > 3:
    #     md += 'location: "' + str(item.location) + '"\n'

    # Add tags if applicable
    if len(str(item.tags)) > 3:
        tags = item.tags.split(",")
        md += "tags: \n"
        for tag in tags:
            md += "  - " + tag + " \n"
    
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_posts/" + md_filename, 'w') as f:
        f.write(md)